<h1> IMPORT </h1>

In [1]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
from threading import Thread

In [2]:
# UTILS FUNCTION
def _getRidOfNonAlphaNumeric (string):
    return re.sub('[^0-9a-zA-Z]+', '', string)
    
def _removeSpecialCarFromText(text):
    return text.replace(u'\t', u'').replace(u'\r', u'').replace(u'\u0153', u'oe').replace(u'\xc2', u'').replace(u'\xa0', u'').replace(u'\xe2', u'').replace(u'\x82', u'').replace(u'\xac', u'').replace(u'\u20ac', u'').replace(u'\n', u'').replace(u'\xc0', u'A').replace(u'\xe9', u'e').replace(u'\xe0', u'a').replace(u'\xe7',u'c').replace(u'\xe8',u'e').replace(u'\xf4',u'o').replace(u'\xee',u'i').replace(u'\xc9',u'E').replace(u'\xd4',u'O').replace(u'\xea',u'e').replace(u'\xfb',u'u').replace(u'\u2019',u' ')

def _getIndices(my_list, things_to_find):
    return [i for i, x in enumerate(my_list) if any(thing in x for thing in things_to_find)]

def _constructVector(dimensions, data):
    res = [0]*len(dimensions)
    for indice in _getIndices(dimensions, data):
        res[indice] = 1
    return res

<h1> PARSING THE NUMBERS </h1>
<p> On scrape le site de The Numbers pour recuperer le buget des films </p>

In [4]:
urlTheNumbers = 'http://www.the-numbers.com/movie/budgets/all'
nameCsv = 'thenumbers.csv'

soup = BeautifulSoup(requests.get(urlTheNumbers).text.encode('utf-8'), 'html.parser')
columns = ['Release Date', 'Movie', 'Production Budget', 'Domestic Gross', 'Worldwide Gross']
df = pd.DataFrame(columns=columns)
nbFilms = 5028
for i in range(2, nbFilms*2 + 1, 2):
    year = soup.select('table tr:nth-of-type(' + str(i) + ') td:nth-of-type(2)')[0].find('a').next
    movie = _removeSpecialCarFromText(soup.select('table tr:nth-of-type(' + str(i) + ') td:nth-of-type(3)')[0].find('a').next)
    prodBudget = _getRidOfNonAlphaNumeric(soup.select('table tr:nth-of-type(' + str(i) + ') td:nth-of-type(4)')[0].next)
    domesticGross = _getRidOfNonAlphaNumeric(soup.select('table tr:nth-of-type(' + str(i) + ') td:nth-of-type(5)')[0].next)
    worldwideGross = _getRidOfNonAlphaNumeric(soup.select('table tr:nth-of-type(' + str(i) + ') td:nth-of-type(6)')[0].next)
    df.loc[i/2-1] = [year, movie, prodBudget, domesticGross, worldwideGross]
    if i % 300 == 0:
        print 'Save ' + str(i)
        df.to_csv(nameCsv, encoding='utf-8')

df.to_csv(nameCsv, encoding='utf-8')

<p> NB tickets vendus et sommes recu par film par annee </p> 

In [ ]:
urlTheNumbers = 'http://www.the-numbers.com/market/{year}/top-grossing-movies'
years = range(1995, 2016)
nameCsv = 'thenumberTopGrossingMovies.csv'

columns = ['Year', 'Rank', 'Movie', 'Release Date', 'Distributor', 'Genre', 'MPAA', 'Gross', 'Tickets Sold']
df = pd.DataFrame(columns=columns)
cpt = 1
for year in years:
    url = urlTheNumbers.format(year=year)
    htmlUrl = requests.get(url).text.encode('utf-8')
    soup = BeautifulSoup(htmlUrl, 'html.parser')
    firstLoop = True
    for row in soup.findAll('tr'):
        if firstLoop:
            firstLoop = False
        else:
            ending = re.search(r'Total Gross of All Movies', row.text)
            if ending is None:
                res = [year]
                for cell in row.findAll('td'):
                    hasLink = cell.find('a')
                    if hasLink is None:
                        res.append(cell.next)
                    else:
                        res.append(hasLink.next)
                res[-1] = int(_getRidOfNonAlphaNumeric(res[-1]))
                res[-2] = int(_getRidOfNonAlphaNumeric(res[-2]))
                res[2] = _removeSpecialCarFromText(res[2])
                res[3] = _removeSpecialCarFromText(res[3])
                df.loc[cpt] = res
                cpt += 1
                if cpt % 100 == 0:
                    print 'Save : ' + str(cpt)
                    df.to_csv(nameCsv, encoding='utf-8')    
            else:
                break
df.to_csv(nameCsv, encoding='utf-8')

<p> On merge les deux dataframe et on les regroupes par films </p>

In [ ]:
# Merge
df1 = pd.read_csv('thenumbers.csv')
df2 = pd.read_csv('thenumberTopGrossingMovies.csv')
df = pd.merge(df1, df2, left_on=['Movie', 'Release Date'], right_on=['Movie', 'Release Date'])

# Clean
df['Sum Tickets Sold Gross'] = df['Gross'].groupby(df['Movie']).transform('sum')
df['Rentabilite Tickets'] = df['Sum Tickets Sold Gross'] / df['Production Budget']
df['Rentabilite'] = df['Worldwide Gross'] / df['Production Budget']
df = df.drop_duplicates('Movie')
df = df[['Movie', 'Rentabilite', 'Rentabilite Tickets']]
df.to_csv('cleanAllMovies.csv')

<p> Creation des colonnes <p>

In [5]:
COLUMNS_MOVIES = ['Movie', 'ratingValue', 'ratingCount', 'reviewUser', 'reviewCritic', 'genreMovie', 'acteurs', 'description']

<p> Parsing de IMDB pour recuperer pour chacun de nos films les notes et les acteurs ainsi qu un resume </p>

In [ ]:
def removeSpecialCarFromText(text):
    return text.replace(u',', u'').replace(u' user',u'').replace(u' critic', '').replace(u' ', u'')#.replace(u'\r', u'').replace(u'\u0153', u'oe').replace(u'\xc2', u'').replace(u'\xa0', u'').replace(u'\xe2', u'').replace(u'\x82', u'').replace(u'\xac', u'').replace(u'\u20ac', u'').replace(u'\n', u'').replace(u'\xc0', u'A').replace(u'\xe9', u'e').replace(u'\xe0', u'a').replace(u'\xe7',u'c').replace(u'\xe8',u'e').replace(u'\xf4',u'o').replace(u'\xee',u'i').replace(u'\xc9',u'E').replace(u'\xd4',u'O').replace(u'\xea',u'e').replace(u'\xfb',u'u').replace(u'\u2019',u' ')

def getSoupFromUrl(url):
    request = requests.get(url)
    return BeautifulSoup(request.text, 'html.parser')

def buildURL(movie_name):
    return 'http://www.imdb.com/find?ref_=nv_sr_fn&q=' + movie_name + '&s=all'

def getURLMovie(soup):
    if soup.find('table', {'class' : 'findList'}) is not None:
        hrefMovies = soup.find('table', {'class' : 'findList'}).findAll('tr')
        hrefMovie = ''
        for mov in hrefMovies:
            if len(mov.findAll('td')) > 1:
                title = mov.findAll('td')[1].text.strip()
                if title.count('(') == 1 and title.count(')') == 1:
                    hrefMovie = mov.findAll('td')[1].find('a')['href']
                    break
            else:
                hrefMovie = ''
    else:
        hrefMovie = ''
    return 'http://www.imdb.com' + hrefMovie

def getInformationFromMovie(soup_movie):
    
    # GET NOTATION OF THE MOVIE
    if (soup_movie.find('span', itemprop='ratingValue') is not None):
        ratingValue = float(soup_movie.find('span', itemprop='ratingValue').text)
    else:
        ratingValue = 0
    if (soup_movie.find('span', itemprop='ratingCount') is not None):
        ratingCount = int(removeSpecialCarFromText(soup_movie.find('span', itemprop='ratingCount').text))
    else:
        ratingCount = 0
    if (soup_movie.findAll('span', itemprop='reviewCount') is not None):
        list_reviews = soup_movie.findAll('span', itemprop='reviewCount')
        if (len(list_reviews) > 1):
            reviewUser = int(removeSpecialCarFromText(list_reviews[0].text))
            reviewCritic = int(removeSpecialCarFromText(list_reviews[1].text))
        else:
            reviewUser = 0
            reviewCritic = 0
    else:
        list_reviews = []
        reviewUser = 0
        reviewCritic = 0
    
    # GENRE MOVIE
    if (soup_movie.find('div', itemprop='genre') is not None):
        genreSoup = soup_movie.find('div', itemprop='genre')
        list_genres = genreSoup.findAll('a')
        genreMovie = [removeSpecialCarFromText(genre.text).strip().lower() for genre in list_genres]
    else:
        genreMovie = []
    
    # CASTING LIST
    if (soup_movie.find('div', {'id':'titleCast'}) is not None):
        soup_acteur = soup_movie.find('div', {'id':'titleCast'})
        list_acteurs = soup_acteur.findAll('span', itemprop='name')
        acteurs = [acteur.text.strip().lower() for acteur in list_acteurs]
    else:
        acteurs = []
    
    # DIRECTOR
    if (soup_movie.find('div', itemprop='director') is not None):
        director = soup_movie.find('div', itemprop='director').find('span').text
        acteurs.insert(0, director.strip().lower())
    
    for acteur in acteurs:
        if acteur not in list_acteur_all:
            list_acteur_all.append(acteur)
    
    # MOVIE DESCRIPTION
    if (soup_movie.find('div', itemprop='description') is not None):
        description = soup_movie.find('div', itemprop='description').find('p').text
        description = description.split("Written by")[0]
        description = description.strip()
    else:
        description = ''
    
    return ratingValue, ratingCount, reviewUser, reviewCritic, genreMovie, acteurs, description

# READ CSV MOVIE
csv_file = pd.read_csv('allMovies.csv', sep=',')
# print csv_file.head()

csv_file['Movie'] = csv_file['Movie'].str.replace(r' ', '+')
csv_file['Movie'] = csv_file['Movie'].str.replace(r':', '')
list_of_movies = csv_file['Movie'].unique()


list_acteur_all = []
# CREATE DATAFRAME 
df_movie = pd.DataFrame(columns=COLUMNS_MOVIES)
indice = 0

<p> Utilisation des threads pour faire le parsing sur IMDB </p>

In [ ]:
class DisplayThread(Thread):

    #Constructeur
    def __init__(self, numThread, movies):
        Thread.__init__(self)
        self.numThread = numThread
        self.movies = movies

    #On definit notre run
    def run(self):
        #On recupere le nb de films traites par thread
        movie_by_thread = len(self.movies)/nb_thread
        #L indice du premier film traite
        first_movie = self.numThread * movie_by_thread
        #L indice du dernier
        last_movie = (self.numThread + 1) * movie_by_thread
        #On effectue notre parsing sur cette partie de films
        indice = self.numThread * movie_by_thread
        for movie in self.movies[first_movie:last_movie]:
            url = buildURL(movie)
            soup = getSoupFromUrl(url)
            url_movie = getURLMovie(soup)

            if url_movie is not '':
                soup_movie = getSoupFromUrl(url_movie)

                ratingValue, ratingCount, reviewUser, reviewCritic, genreMovie, acteurs, description = getInformationFromMovie(soup_movie)

                df_movie.loc[indice] = [movie, ratingValue, ratingCount, reviewUser, reviewCritic, genreMovie, acteurs, description]

                if (indice % 10) == 0:
                    print 'Save ' + str(indice)
                    df_movie.to_csv('movieRate.csv', encoding='utf-8')
            print indice
            indice+=1

nb_thread = os.sysconf("SC_NPROCESSORS_ONLN") * 2
print nb_thread
threads = []
for i in range(0,nb_thread):
    t = DisplayThread(i, list_of_movies)
    threads.append(t)
    t.start()